In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import pandas as pd

from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [11]:
# lendo dataset
df = pd.read_csv('DataSet1.csv')
df.head()

,RSSI,distancia
0,-43,"0,5"
1,-47,"0,5"
2,-47,"0,5"
3,-46,"0,5"
4,-45,"0,5"


In [ ]:
# normalizacao dos dados
# dfNormalizado = minMax
scaler = MinMaxScaler(feature_range=(-1, 1))
dfNormalizado = scaler.fit_transform(df).tolist()

In [15]:
# separacao dataset

train, test = train_test_split(dfNormalizado, test_size=0.15, shuffle=True)
train, val = train_test_split(train, test_size=0.18, shuffle=True)

In [16]:
# a função unstitch separa os atributos da label
def unstitch(data):
    df = pd.DataFrame(data)
    return df[0].values.tolist(), df[1].values.tolist()

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, 1)
        )
# sigmoid ou relu ou tanh
                

        #a função de loss utilizada é MSE
        self.criterion = nn.MSELoss()
        # #o otimizador selecionado foi SGD
        self.optimizer = optim.SGD(self.parameters(), lr=learning_rate)
   
   
    def forward(self, x):
        # função de foward
        return self.layers(x)
    
    
    
    
    def loss(self, loss_set):
        #a função de loss é definida
        inputs, labels = unstitch(loss_set)
        inputs = Variable(torch.IntTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))
        result = self(inputs)
        loss = self.criterion(result, torch.max(labels, 1)[1])

        return float(loss.item())
    

    def accuracy(self, acc_set):
        #definição do cálculo da acurácia
        inputs, labels = unstitch(acc_set)
        result = self(Variable(torch.FloatTensor(inputs)))
        inputs_max = np.argmax(result.detach().cpu().numpy(), axis=1)
        labels_max = np.argmax(np.array(labels), axis=1)
        correct = np.sum(inputs_max == labels_max)

        return correct/len(acc_set)


    def train(self, numEpochs, train):
        #função que realiza o treino da rede
        #o treino inicia com a separação dos parâmetros e das labels
        inputs, labels = unstitch(train)
        
        inputs = Variable(torch.IntTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))#isso tranfoma pra float os dados?

        #em seguida, para cada época é realizado o forward propagation, e então o otimizador
        #efetua o back propagation atualizando os pesos de cada neurônio
        #os resultados de cada época são armazenados para exibição no TensorBoard
        for epoch in range(numEpochs):
            self.optimizer.zero_grad()
            outputs = self(inputs)
            self.loss = self.criterion(outputs, labels)
            self.loss.backward()    
            self.optimizer.step()

            self.store_results(epoch, train)
            self.print_final_train_acc(epoch, train)
  

In [7]:
# HYPERPARAMETROS

learning_rate = 0.0001
batch_size = 32
num_epochs = 5

batchNet = Net()
batchNet.train(num_epochs, train)
  
 #a acurácia de validação é exibida
print(f"Validation accuracy: {batchNet.accuracy(val)}")
# acurácia
#para o conjunto de teste
print(f"Test accuracy: {batchNet.accuracy(test)}")

NameError: name 'Net' is not defined